In [6]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import time
import importlib
from pathlib import Path
from datetime import datetime

from numba import jit
import numpy as np
import pandas as pd

from neuropy import core

from pyphocorehelpers.preferences_helpers import set_pho_preferences
set_pho_preferences() # Set preferences
# from pyphoplacecellanalysis.General.SpikesDataframeWindow import SpikesDataframeWindow, SpikesWindowOwningMixin
# from pyphoplacecellanalysis.General.DataSeriesToSpatial import DataSeriesToSpatial
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster_Vedo import Spike3DRaster_Vedo

%gui qt5
# curr_epoch_name = 'maze1'
# curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
# curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
# curr_spikes_df = curr_sess.spikes_df

from vedo import show

def trap_exc_during_debug(*args):
    # when app raises uncaught exception, print info
    print(args)


# install exception hook: without this, uncaught exception would cause application to exit
sys.excepthook = trap_exc_during_debug



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Load the saved .h5 spikes dataframe for testing:
finalized_spike_df_cache_file='./pipeline_cache_store.h5'
desired_spikes_df_key = '/filtered_sessions/maze1/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
curr_spikes_df

print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}")

aclu:   	[ 2  3  4  5  7  8  9 10 11 12 14 17 18 21 22 23 24 25 26 27 28 29 33 34 38 39 42 44 45 46 47 48 53 55 57 58 61 62 63 64]
unit_id:	[ 0  1  2  3  5  6  7  8  9 10 12 15 16 19 20 21 22 23 24 25 26 27 31 32 36 37 40 42 43 44 45 46 51 53 55 56 59 60 61 62]


In [15]:
desired_spikes_df_key = '/sess/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
curr_spikes_df

print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}")

aclu:   	[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65]
unit_id:	[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


In [13]:
desired_spikes_df_key = '/filtered_sessions/maze2/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}") # note unit_id can be missing indicies because each spike might not have an entry in it

aclu:   	[ 2  3  4  5  7  8  9 10 11 12 14 17 18 21 22 23 24 25 26 27 28 29 33 34 38 39 42 44 45 46 47 48 53 55 57 58 61 62 63 64]
unit_id:	[ 0  1  2  3  5  6  7  8  9 10 12 15 16 19 20 21 22 23 24 25 26 27 31 32 36 37 40 42 43 44 45 46 51 53 55 56 59 60 61 62]


In [2]:

def add_filtered_session_cell_idx_column(spikes_df):
    """ 
    
    """
    if 'filtered_session_cell_idx' not in spikes_df.columns:
        # self.spikes_df['cell_idx'] = self.spikes_df['unit_id'].copy() # TODO: this is bad! The self.get_neuron_id_and_idx(...) function doesn't work!
        # note that this is very slow, but works:
        print(f'cell_idx column missing. rebuilding (this might take a minute or two)...')
        included_cell_INDEXES = np.array([self.get_neuron_id_and_idx(neuron_id=an_included_cell_ID)[0] for an_included_cell_ID in spikes_df['aclu'].to_numpy()]) # get the indexes from the cellIDs
        spikes_df['filtered_session_cell_idx'] = included_cell_INDEXES.copy()

,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,...,maze_relative_lap,maze_id,cell_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,unit_id,PBE_id,scISI
12517,725979.0,506939.938112,22.302063,7,7,42,4,228.439138,145.608734,4.518929,...,-1,1,NeuronType.PYRAMIDAL,12517,0.793762,0.498754,-46.025885,40,-1,0.715745
12520,726167.0,506939.943888,22.307839,2,5,14,4,228.428329,145.632192,4.517234,...,-1,1,NeuronType.PYRAMIDAL,12520,0.793826,0.498658,-46.015141,12,-1,12.857849
12532,726835.0,506939.964409,22.328360,2,28,27,4,228.389921,145.715544,4.511208,...,-1,1,NeuronType.PYRAMIDAL,12532,0.794084,0.498267,-45.976965,25,-1,0.141711
12627,731665.0,506940.112786,22.476737,11,2,58,4,228.110399,146.322156,4.495592,...,-1,1,NeuronType.PYRAMIDAL,12627,0.795136,0.497673,-45.699695,56,-1,0.510628
12633,732082.0,506940.125597,22.489548,8,13,53,4,228.085786,146.375570,4.484851,...,-1,1,NeuronType.PYRAMIDAL,12633,0.795016,0.497707,-45.675124,51,-1,5.395750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831590,56605920.0,508656.569021,1738.932972,2,20,21,4,212.373363,138.488006,15.584255,...,-1,1,NeuronType.PYRAMIDAL,831590,0.766190,0.478007,-29.938979,19,-1,0.002826
831601,56607201.0,508656.608373,1738.972324,2,20,21,4,211.783865,138.621493,15.438952,...,-1,1,NeuronType.PYRAMIDAL,831601,0.763523,0.477928,-29.351434,19,-1,0.039352
831623,56609125.0,508656.667479,1739.031430,11,2,58,4,210.881567,138.825812,15.409233,...,-1,1,NeuronType.PYRAMIDAL,831623,0.759478,0.477808,-28.475510,56,-1,0.186347
831662,56612413.0,508656.768486,1739.132437,2,20,21,4,209.352483,139.172060,15.495960,...,-1,1,NeuronType.PYRAMIDAL,831662,0.752613,0.477605,-27.286186,19,-1,0.160113


# Vedo 3D Spike Train Visualization 

In [ ]:
# Build the 3D Raster object:
spike_raster_plt = Spike3DRaster_Vedo(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)


In [ ]:
spike_raster_plt._update_plots()
axes = dict(xtitle='window time', ytitle='cell_id', ztitle="")
# plt = show(spike_raster_plt.glyph, __doc__, axes=axes, interactive=False, bg='k8', backend='ipyvtk') # plt: <vedo.plotter.Plotter at 0x22eb25de370>
# plt = show(spike_raster_plt.glyph, __doc__, axes=axes, interactive=True, bg='k8', backend='ipyvtk') # crashes when interactive=True

# Pyqtgraph 3D Spike Train Visualization

In [ ]:
spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)
# spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=0.2, window_start_time=30.0)

In [ ]:
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')
# spike_raster_plt.spikes_window.active_window_start_time = 50.0
spike_raster_plt.spikes_window.update_window_start(90.0)
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')